# Machine Learning Basics with Scikit-learn: Day 5
In our last session, we will build a model to predict the chronic kidney disease. The data was taken over a 2-month period in India with 25 features ( eg, red blood cell count, white blood cell count, etc). The target is the 'classification', which is either 'ckd' or 'notckd' - ckd=chronic kidney disease. We will use machine learning techniques to predict if a patient is suffering from a chronic kidney disease or not. This dataset has misssing data, features in different scales, and 25 features. For more information click [here](https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease). 

We will perform the following actions:
1. Import the dataset
2. Analyze the dataset
3. Imput missing values
4. Scale the data
5. Create training and testing datasets
6. Train a neural network with the training dataset
7. Evaluate the model with the testing dataset

Let's start loading the functions and libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.impute import SimpleImputer

# 1. Import the dataset

We import the dataset. This time, we will use a CSV file which contains the required data.

In [ ]:
kidney_df = pd.read_csv("https://raw.githubusercontent.com/dgzara/nuit_machine_learning_workshop/main/kidney_disease.csv", index_col=0)
kidney_df

The target column was coded using strings (`notckd` and `ckd`). We can replace them with binary values (0 and 1).  

In [ ]:
kidney_df['classification'] = kidney_df['classification'].str.replace('notckd', '0')
kidney_df['classification'] = kidney_df['classification'].str.replace('ckd', '1')

In [ ]:
kidney_df

We split the dataset into `X` and `y` variables

In [ ]:
X = kidney_df.drop(['classification'], axis=1)
y = kidney_df['classification'].astype(int)

## 2. Analyze the dataset

According to the dataset's documentation, the features available are:
1. age - age
2. bp - blood pressure
3. sg - specific gravity
4. al - albumin
5. su - sugar
6. rbc - red blood cells
7. pc - pus cell
8. pcc - pus cell clumps
9. ba - bacteria
10. bgr - blood glucose random
11. bu - blood urea
12. sc - serum creatinine
13. sod - sodium
14. pot - potassium
15. hemo - hemoglobin
16. pcv - packed cell volume
17. wc - white blood cell count
18. rc - red blood cell count
19. htn - hypertension
20. dm - diabetes mellitus
21. cad - coronary artery disease
22. appet - appetite
23. pe - pedal edema
24. ane - anemia

We check the columns:

In [ ]:
X.columns

Are the columns numeric or categorical?

In [ ]:
X.dtypes

In [ ]:
# How much data is missing? 
X.isna().sum()

In [ ]:
# What are the main descriptive statistics of this dataset?
X.describe()

The first finding is that some columns have many missing observations. For example, potassium (`pot`) has 88 missing values. 

How is the numeric data distributed?

In [ ]:
X.hist(figsize=(15,10))
plt.show()

We are going to select the numeric columns and categorical columns separately for our transformations

In [ ]:
numeric_columns = X.select_dtypes(include=np.number).columns.tolist()
numeric_columns

In [ ]:
categorical_columns = list(set(X.columns) - set(numeric_columns))
categorical_columns

## 3. Imput missing values

We will use the `SimpleImputer` class, which provides basic strategies for imputing missing values. Missing values can be imputed with a provided constant value, or using the statistics (mean, median or most frequent) of each column in which the missing values are located. This class also allows for different missing values encodings.
* For numeric attributes, we can use `mean` to replace `NaN` values.
* For categorical attributes, we can use `most_frequent` to replace `NaN` values.

Let's start imputing the missing numerical data

In [ ]:
# We will copy the X dataframe to X_imp and keep the original dataframe X separated
X_imp = X.copy()

In [ ]:
# Create the SimpleImputer object
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# Use the imputer to fit and transform the numeric data. Replace the 
X_imp[numeric_columns] = imp.fit_transform(X_imp[numeric_columns])

In [ ]:
X_imp[numeric_columns]

In [ ]:
# Check how many missing values each feature has. 
X_imp.isna().sum()

### Exercise
Create an `SimpleImputer` object and impute the missing categorical data. Use `most_frequent` as the strategy. Do not forget to use `X_imp` for this operation:

In [ ]:
# Create the imputer using `most_frequent` strategy
# imp = ....

In [ ]:
# Impute the missing categorical values of X_imp
# X_imp[categorical_columns] = ...

In [ ]:
# Check the results
# X_imp[categorical_columns]

In [ ]:
# Check how many missing values each feature has. All values should be zero now. 
# X_imp.isna().sum()

## 4. Scale the data
Yesterday we discussed the importance of scaling numeric data to (a) handle outliers and (b) compare the effect of increasing one unit of each feature on the dependent variable. We will standarize the numeric features present in this dataset.

In [ ]:
# We copy the imputed data to a new dataframe.
X_scaled = X_imp.copy()

### Exercise
Create the scaler using `preprocessing.StandardScaler()` and then replace the numeric columns of `X_scaled` with the scaler.

In [ ]:
# We create the scaler using preprocessing.StandardScaler() function
# scaler = ...

In [ ]:
# We use the scaler to fit and transform only the numeric features (`X_imp[numeric_columns]`)
# X_scaled[numeric_columns] = ....

In [ ]:
# We print the descriptive statistics
# round(X_scaled.describe(),2)

In [ ]:
# We plot the distributions and check the scales
# X_scaled.hist(figsize=(15,10))
# plt.show()

### Handling categorical data
These are transformers that are not intended to be used on features, only on supervised learning targets. For example, categorical data is not mathematically represented in the dataset. 

To handle this situation, scikit-learn provides the function `LabelEncoder`, which is a utility class to help normalize labels from categorical features. It will replace the classes with integers that represent each class. In R, this is also known as *factors.* `LabelEncoder` can be used as follows:

In [ ]:
# We create the `LabelEncoder`
le = preprocessing.LabelEncoder()

In [ ]:
# Create a toy example. First, we will encode the classes and detect the unique classes
le.fit(["low", "medium", "high", "medium"])
list(le.classes_)

In [ ]:
# Based on this Encoder, we took the vector of categorical data and transform it to numeric data
le.transform(["low", "low", "high"])

In [ ]:
# We can decode a vector of numeric values to obtain the original classes
list(le.inverse_transform([1, 1, 0]))

Now, let's get back to our dataset. We need to transform the categorical features and encode them:

In [ ]:
categorical_columns

### Exercise
Since we have 13 categorical features, we will create a Python loop and save the encoders in a dictionary. You will have to create inside of the loop the Encoder and transform the dataset.

In [ ]:
# Create a dictionary of encoders
encoders = {}

# Run the loop
for column in categorical_columns:
    # Create the LabelEncoder
    # le = ....
    
    # Train the LabelEncoder using the X[column] 
    # 
    
    # Replace the X_scale[column] with the transformed data
    # X_scaled[column] = ...
    
    # Append the Encoder to the dictionary
    # encoders[column] = le

In [ ]:
# Check the dictionary of encoders
# encoders

In [ ]:
# We check the columns, we will see that now all columns are numeric.
# X_scaled.dtypes

In [ ]:
# We can print the final dataframe
# X_scaled

## 5. Create training and testing datasets
We split our dataset into two portions: training dataset and testing dataset. We will use the former to train the predictive model, and the latter to evaluate the model's accuracy. To split the data, we use `train_test_split` function provided by scikit-learn library. There are four parameters of this function
* The imput data (X_scaled)
* The target vector (y)
* `test_size`: The dataset's percentage that will be used for testing. We will use `0.20`.
* `random_state`: Random seed. We can keep it fixed so we can reproduce the random numbers across multiple calls. We will use [42](https://hitchhikers.fandom.com/wiki/42). 

### Excercise
Create the training and testing datasets. Remember that this method returns 4 variables (`X_train`, `X_test`, `y_train`, `y_test`). You can check the function's [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to check how the parameters are set up.

In [ ]:
# Create here the training and testing datasets using train_test_split
# X_train, X_test, y_train, y_test = ...

In [ ]:
# We print the training dataset. We will have 320 observations.
# X_train

In [ ]:
# Check the testing dataset. We will have 80 observations
# X_test

## 6. Train a neural network with the training dataset

For this exercise, we will create a neural network called Multi-layer perceptron (MLP). The model learns from the dataset through three types of layers:
* *The input layer*: The input layer receives the features to be processed.
* *The output layer*: The prediction and classification is performed by the output layer.
* *The hidden layers*: An arbitrary number of hidden layers that are placed in between the input and output layer perform the computation. Each layer has nodes (also known as neurons) that transforms the values from the previous layer into an activation funcition. The output layer receives the activated values from the last hidden layer and transforms them into output values.

<img src="https://static.javatpoint.com/tutorial/tensorflow/images/multi-layer-perceptron-in-tensorflow.png" alt="drawing" width="400"/>

Multi-layer perceptrons are networks of nodes that pass a linear combination of their inputs from one layer to another. As they do this, the nodes decide how to modify their inputs, utilizing a given activation function. The activation function of a neuron is the key here. By selecting non-linear activation functions, the neural network can embed non-linearity in its operation:

<img src="https://www.baeldung.com/wp-content/uploads/sites/4/2020/06/Copy-of-Copy-of-Blank-Diagram1.svg" alt="drawing" width="400"/>

Neural networks are good for non-linear classes of problems. The first advantage of neural networks is, therefore, their flexibility in addressing problems with non-linear shapes. MLPs are implemented in diverse fields such as speech recognition, image recognition, and machine translation software. From artificial neural networks, **deep learning** has emerged as a sub-field of machine learning.

Let's create a simple perceptron with 5 nodes in two layers

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)

### Exercise
Use the `fit` function to train the model. Remember that it has two parameters: `X_train` and `y_train`.
Then, use the `predict` to predict the values using `X_test`.

In [ ]:
# ...

In [ ]:
# Use the predict function to test the model
# y_predicted = ...

In [ ]:
# We create a dataframe with the real target and the predicted target
# y_train_df = pd.DataFrame({'y_real': y_test, 'y_predicted': y_predicted})
# y_train_df['difference'] = y_train_df.y_real == y_train_df.y_predicted 
# y_train_df

## 7. Evaluate the model with the testing dataset

In binary classiifcation problems, there are two main metrics:
* Precision: What proportion of positive identifications was actually correct?
* Recall: What proportion of actual positives was identified correctly?

Precision and recall are performance metrics that apply to data retrieved from a classification process. We can see the diferences here:

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/700px-Precisionrecall.svg.png" width="400px">


To check the mathematical formulas, check [here](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall).

In [ ]:
# We a text report showing the main classification metrics.
# print(classification_report(y_test, y_predicted))

In [ ]:
# We can print these numbers as the confusion matrix.
# plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
# plt.show()

### Exercise
Create a new model using 10 layers with 10 nodes each layer. Would the performance improve? 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=42)

In [ ]:
# Create the model, fit, and predict.
#clf = ....

In [ ]:
# print(classification_report(y_test, y_predicted))

In [ ]:
# plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
# plt.show()  

## What if...

### Using the raw data, removing the NaNs and only numerical values

In [ ]:
kidney_no_na = kidney_df.dropna()
X_no_na = kidney_no_na.drop(['classification'], axis=1)
y_no_na = kidney_no_na['classification'].astype(int)

In [ ]:
X_no_na

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_no_na[numeric_columns], y_no_na, test_size=0.20, random_state=42)

In [ ]:
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
plt.show()  

### No categorical attributes and no rescale

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_imp[numeric_columns], y, test_size=0.20, random_state=42)

In [ ]:
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=1000)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
plt.show()  

### Scaled, but no categorical attributes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled[numeric_columns], y, test_size=0.20, random_state=42)

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(5, 2), random_state=1, max_iter=1000)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
plt.show()  

### Only categorical variables

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled[categorical_columns], y, test_size=0.20, random_state=42)

In [ ]:
clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1,max_iter=2000)
clf.fit(X_train, y_train)
y_predicted = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues)  
plt.show()  

## More to learn...
* How do we estimate the correct parameters to use?: [Hyperparameter optimization](https://scikit-learn.org/stable/modules/grid_search.html)
* How do evaluate the model? [Metrics and scoring](https://scikit-learn.org/stable/modules/model_evaluation.html)
* How can I train a model using different partitions of my dataset? [Corss-validation](https://scikit-learn.org/stable/modules/cross_validation.html)
* Neural Networks: [TensorFlow](https://www.tensorflow.org/)